In [23]:
#Constants:
c_train_df_prep_file='train_df_prep_file.pkl'
c_bst_file='bst.pkl'

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import lightgbm as lgb
import gc

In [25]:
metrics = 'auc'

In [26]:
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric':metrics,
        'learning_rate': 0.1,
        'num_leaves': 7,  # we should let it be smaller than 2^(max_depth)
        'max_depth': 4,  # -1 means no limit
        'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 100,  # Number of bucketed bin for feature values
        'subsample': 0.7,  # Subsample ratio of the training instance.
        'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.7,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'nthread': 4,
        'verbose': 0,
        'scale_pos_weight':99.7, # because training data is extremely unbalanced 
        'metric':metrics
}
target = 'is_attributed'
predictors = ['app','device','os', 'channel', 'hour']#,'nip_day_test_hh', 'nip_day_hh',
#               'nip_hh_os', 'nip_hh_app', 'nip_hh_dev'
categorical = ['app', 'device', 'os', 'channel', 'hour']

In [28]:
train_df_prep=pd.read_pickle(c_train_df_prep_file)
# train_df, val_df = train_test_split( train_df_prep, train_size=.95, random_state=99, shuffle=True )
 
print(train_df.info())
print(val_df.info())

print("train size: ", len(train_df))
print("valid size: ", len(val_df))
del train_df_prep
gc.collect()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 7 columns):
app              750000 non-null uint16
device           750000 non-null uint16
os               750000 non-null uint16
channel          750000 non-null uint16
is_attributed    750000 non-null uint8
hour             750000 non-null uint8
in_test_hh       750000 non-null uint8
dtypes: uint16(4), uint8(3)
memory usage: 7.9 MB
None


NameError: name 'val_df' is not defined

In [29]:
VALIDATE = True
MAX_ROUNDS = 500
EARLY_STOP = 50
OPT_ROUNDS = 500
print("Training...")

num_boost_round=MAX_ROUNDS
early_stopping_rounds=EARLY_STOP

xgtrain = lgb.Dataset(train_df[predictors].values, label=train_df[target].values,
                      feature_name=predictors,
                      categorical_feature=categorical
                      )
del train_df
gc.collect()
print('done...')

Training...
done...


In [20]:
xgvalid = lgb.Dataset(val_df[predictors].values, label=val_df[target].values,
                      feature_name=predictors,
                      categorical_feature=categorical
                      )
del val_df
gc.collect()

93

In [ ]:
evals_results = {}
bst = lgb.train(lgb_params, 
                 xgtrain, 
                 valid_sets=[xgtrain], 
                 valid_names=['train'], 
                 evals_result=evals_results, 
                 num_boost_round=num_boost_round,
                 early_stopping_rounds=early_stopping_rounds,
                 verbose_eval=10, 
                 feval=None)

n_estimators = bst.best_iteration
bst.save_model(c_bst_file)
print("\nModel Report")
print("n_estimators : ", n_estimators)
# print(metrics+":", evals_results['valid'][metrics][n_estimators-1])
# del xgvalid

/home/ubuntu/anaconda3/envs/dog-project/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[10]	train's auc: 0.969306
[20]	train's auc: 0.971329
[30]	train's auc: 0.973831
[40]	train's auc: 0.975867
[50]	train's auc: 0.977808
[60]	train's auc: 0.979183
[70]	train's auc: 0.98051
[80]	train's auc: 0.981669
[90]	train's auc: 0.982714
[100]	train's auc: 0.983522
[110]	train's auc: 0.984154
[120]	train's auc: 0.984661
[130]	train's auc: 0.985226
[140]	train's auc: 0.985785
[150]	train's auc: 0.986102
[160]	train's auc: 0.986458
[170]	train's auc: 0.986754
[180]	train's auc: 0.986853
[190]	train's auc: 0.987116
[200]	train's auc: 0.987241
[210]	train's auc: 0.987399
[220]	train's auc: 0.987587
[230]	train's auc: 0.987624
[240]	train's auc: 0.987685
[250]	train's auc: 0.987844
[260]	train's auc: 0.988015
[270]	train's auc: 0.988137
[280]	train's auc: 0.988152
[290]	train's auc: 0.988257
[300]	train's auc: 0.988235
[310]	train's auc: 0.988364
[320]	train's auc: 0.988509
[330]	train's auc: 0.988499
[340]	train's auc: 0.988

In [21]:
evals_results = {}
bst = lgb.train(lgb_params, 
                 xgtrain, 
                 valid_sets=[xgtrain, xgvalid], 
                 valid_names=['train','valid'], 
                 evals_result=evals_results, 
                 num_boost_round=num_boost_round,
                 early_stopping_rounds=early_stopping_rounds,
                 verbose_eval=10, 
                 feval=None)

n_estimators = bst.best_iteration
bst.save_model(c_bst_file)
print("\nModel Report")
print("n_estimators : ", n_estimators)
print(metrics+":", evals_results['valid'][metrics][n_estimators-1])
del xgvalid

/home/ubuntu/anaconda3/envs/dog-project/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/ubuntu/anaconda3/envs/dog-project/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 50 rounds.
[10]	train's auc: 0.969211	valid's auc: 0.950359
[20]	train's auc: 0.971378	valid's auc: 0.955819
[30]	train's auc: 0.97418	valid's auc: 0.964316
[40]	train's auc: 0.976882	valid's auc: 0.964807
[50]	train's auc: 0.978825	valid's auc: 0.966639
[60]	train's auc: 0.980549	valid's auc: 0.962793
[70]	train's auc: 0.981846	valid's auc: 0.965478
[80]	train's auc: 0.982939	valid's auc: 0.963937
[90]	train's auc: 0.98364	valid's auc: 0.963965
[100]	train's auc: 0.984318	valid's auc: 0.965378
Early stopping, best iteration is:
[50]	train's auc: 0.978825	valid's auc: 0.966639

Model Report
n_estimators :  50
auc: 0.9666387484993704
